In [ ]:
import os
import numpy as np
import pandas as pd
import time, datetime

import matplotlib.pyplot as plt




from sklearn.cluster import KMeans
# from sklearn.cluster import AffinityPropagation
# from sklearn.cluster import AgglomerativeClustering
# from sklearn.cluster import MeanShift





In [ ]:
curr_wd = os.getcwd()
data_dir = os.path.join(curr_wd, 'Data')
data_dir

In [ ]:
demand_raw_dir = os.path.join(data_dir, 'SAMPLE3YearSales.txt')
demand_raw_df = pd.read_csv(demand_raw_dir, sep = '\t', header=0)
print (np.size(demand_raw_df, 0))
demand_raw_df.head()

In [ ]:
add_raw_demand_dir = os.path.join(data_dir, 'SAMPLEWeek1_4.txt')
add_raw_demand_df = pd.read_csv(add_raw_demand_dir, sep='\t', header=0)
add_raw_demand_df.head()

In [ ]:
format_add = add_raw_demand_df.sort_values(['Subclass', 'SKU', 'Store', 'Week'])
LIST = format_add.loc[:, ['SKU', 'Store']].drop_duplicates().copy().reset_index(drop=True)

add_sales = format_add[(format_add.Week >= 201701)]['Units'].values.reshape(-1,add_raw_demand_df.Week.nunique())
add_sales_df = pd.DataFrame(add_sales, columns = [i for i in range(1, add_raw_demand_df.Week.nunique()+1)])

add_df = pd.concat([LIST, add_sales_df], axis = 1)
add_df

In [ ]:
format_df = demand_raw_df.sort_values(['Subclass', 'SKU', 'Store', 'Week'])
STR_SKU_LIST = format_df.loc[:, ['SKU', 'Store']].drop_duplicates().copy().reset_index(drop=True)

sales_2014 = format_df[(format_df.Week < 201501)]['Units'].values.reshape(-1,52)
sales_2015 = format_df[(format_df.Week > 201452) & (format_df.Week < 201601)]['Units'].values.reshape(-1,52)
sales_2016 = format_df[(format_df.Week > 201552) & (format_df.Week < 201701)]['Units'].values.reshape(-1,52)

two_years_mean = (sales_2014 + sales_2015) / 2
three_years_mean = (sales_2014 + sales_2015 + sales_2016) / 3

df_2Years = pd.DataFrame(two_years_mean, columns = [i for i in range(1,53)])
df_3Years = pd.DataFrame(three_years_mean, columns = [i for i in range(1,53)])

df_Year1 = pd.DataFrame(sales_2014, columns = [i for i in range(1,53)])
df_Year2 = pd.DataFrame(sales_2015, columns = [i for i in range(1,53)])
df_Year3 = pd.DataFrame(sales_2016, columns = [i for i in range(1,53)])

two_years_mean_df = pd.concat([STR_SKU_LIST, df_2Years], axis = 1)
three_years_mean_df = pd.concat([STR_SKU_LIST, df_3Years], axis = 1)
year1_df = pd.concat([STR_SKU_LIST, df_Year1], axis = 1)
year2_df = pd.concat([STR_SKU_LIST, df_Year2], axis = 1)
year3_df = pd.concat([STR_SKU_LIST, df_Year3], axis = 1)

In [ ]:
#Functions:

def calc_ncq(df):
    ncq_df = df.iloc[:,2:54].cumsum(axis = 1)
    ncq_max = ncq_df.iloc[:,-1]
    ncq_out =  ncq_df.values /  ncq_max.values[:,None]
    out_df = df.copy()
    out_df.iloc[:,2:54] = ncq_out
    
    return out_df

def scal_qty(df):
    scal_df = df.iloc[:,2:54].copy()
    scal_mean = scal_df.mean(axis = 1)
    scal_out =  scal_df.values /  scal_mean.values[:,None]
    out_df = df.copy()
    out_df.iloc[:,2:54] = scal_out
    
    return out_df

def get_km_result_df(data_ncq, data_scal, kmeans):
    title_df = data_ncq.loc[:,['SKU', 'Store']]
    title_df['Class_label'] = kmeans.labels_
    scal_df = title_df.merge(data_scal, how = 'left', on =['SKU', 'Store'])
    centers_df = scal_df.iloc[:,2:].groupby('Class_label').mean().copy()
    centers_df.reset_index(inplace=True)
    kmean_result_df = title_df.merge(centers_df, how = 'left', on =['Class_label'])
    
    return kmean_result_df

def scal_error_est(km_result, year3_df):
    year3_scal = scal_qty(year3_df)
    compare = km_result.merge(year3_scal, how = 'left', on =['SKU', 'Store'])
    error_matrix = compare.iloc[:,3:55].values - compare.iloc[:,55:107].values
    
    squared_error = np.sum(np.square(error_matrix))
    return squared_error

def ncq_error_est(km_result, year3_df):
    year3_scal = calc_ncq(year3_df)
    km = calc_ncq(km_result.drop(['Class_label'], axis = 1))
    compare = km.merge(year3_scal, how = 'left', on =['SKU', 'Store'])
    error_matrix = compare.iloc[:,2:54].values - compare.iloc[:,54:106].values
    
    squared_error = np.sum(np.square(error_matrix))
    return squared_error

def get_Kmeans_fcst(base_df, factors_df):
    merge = pd.merge(base_df, factors_df.drop(['Class_label'], axis=1), how='inner', on=['SKU', 'Store'])
    matrix_base = merge.iloc[:,2:54].values
    matrix_factors = merge.iloc[:,54:106].values
    matrix_fcst = matrix_base*matrix_factors
    fcst_df = pd.DataFrame(matrix_fcst, columns=['Wk%d' % i for i in range(1,53)])
    result = pd.concat([merge.loc[:,['SKU','Store']], fcst_df], axis =1)
    return result

In [ ]:
#Try to find the best K

#drop rows with any 'null' value 
#drop rows with sum of all sales equal to zero
data_clean = two_years_mean_df.dropna(axis = 0, how = 'any')
data_clean = data_clean[(data_clean.iloc[:,2:54].sum(axis =1) > 0)]

#smooth raw demand using exponential weighted moving average
data_ewma = data_clean.copy()
data_ewma.iloc[:,2:54] = data_clean.iloc[:,2:54].ewm(span = 4, axis = 1).mean()
print('Count of Store_SKU in ewma: %d' % np.size(data_ewma, 0))

#set threshhold for sales volume, item will be grouped together when annully sales <= 52
data_slow_mover = data_ewma[(data_ewma.iloc[:,2:54].sum(axis =1) <= 52)]
data_slow_mover = data_slow_mover.iloc[:, 0:54]
print ('Count of Store_SKU in slow_mover: %d' % np.size(data_slow_mover, 0))

#set threshhold for sales volume, item can be processed when annully sales > 52
data_vol = data_ewma[(data_ewma.iloc[:,2:54].sum(axis =1) > 52)]
print ('Count of Store_SKU to be processed: %d' % np.size(data_vol, 0))


data_ncq = calc_ncq(data_vol)
data_scal = scal_qty(data_vol)


#Finding best K for target dataset
test_list =[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
squared_error = []
for i in test_list:
    print (str(datetime.datetime.now()))
    X_train = data_ncq.iloc[:,2:54].values
    kmeans = KMeans(n_clusters = i, random_state=0).fit(X_train)
    error = ncq_error_est(get_km_result_df(data_ncq, data_scal, kmeans), year3_df)
    squared_error.append(error)
    print (str(datetime.datetime.now()))
    print ("Error for %d is: %s" % (i, error))

print(squared_error)


plt.plot(test_list, squared_error)
plt.show()


In [ ]:
final_k = test_list[squared_error.index(min(squared_error))]
final_k

In [ ]:
#final run with best k


#drop rows with any 'null' value 
#drop rows with sum of all sales equal to zero
data_clean = three_years_mean_df.dropna(axis = 0, how = 'any')
data_clean = data_clean[(data_clean.iloc[:,2:54].sum(axis =1) > 0)]

#smooth raw demand using exponential weighted moving average
data_ewma = data_clean.copy()
data_ewma.iloc[:,2:54] = data_clean.iloc[:,2:54].ewm(span = 4, axis = 1).mean()
print('Count of Store_SKU in ewma: %d' % np.size(data_ewma, 0))

#set threshhold for sales volume, item will be grouped together when annully sales <= 52
data_slow_mover = data_ewma[(data_ewma.iloc[:,2:54].sum(axis =1) <= 52)]
data_slow_mover = data_slow_mover.iloc[:, 0:54]
print ('Count of Store_SKU in slow_mover: %d' % np.size(data_slow_mover, 0))

#set threshhold for sales volume, item can be processed when annully sales > 52
data_vol = data_ewma[(data_ewma.iloc[:,2:54].sum(axis =1) > 52)]
print ('Count of Store_SKU to be processed: %d' % np.size(data_vol, 0))


data_ncq = calc_ncq(data_vol)
data_scal = scal_qty(data_vol)


print (str(datetime.datetime.now()))
X_train = data_ncq.iloc[:,2:54].values
kmeans = KMeans(n_clusters = final_k, random_state=0).fit(X_train)

pieces = []

#get the result from Kmeans
kmeans_result = get_km_result_df(data_ncq, data_scal, kmeans)
pieces.append(kmeans_result)


#get labels for slow movers
scal_slow_mover = scal_qty(data_slow_mover)
scal_slow_mover['Class_label'] = np.array([9999]*np.size(scal_slow_mover, 0))
slow_mover_centers = scal_slow_mover.iloc[:,2:].groupby('Class_label').mean()
slow_mover_centers.reset_index(inplace=True)
slow_mover_result = scal_slow_mover.loc[:,['SKU', 'Store', 'Class_label']].merge(slow_mover_centers, how='left', on=['Class_label'])

pieces.append(slow_mover_result)

df_result = pd.concat(pieces)
df_result['SKU'] = df_result['SKU'].astype('object')
df_result['Store'] = df_result['Store'].astype('object')

print (str(datetime.datetime.now()))

df_result.head()



In [ ]:
#export seasonal factors at str_sku level
df_export = df_result.copy()
df_export.to_csv(os.path.join(data_dir, 'Output/sample_Seasonalities_0109.csv'))
df_export.head()

In [ ]:
a = df_export[(df_export.SKU == 999) & (df_export.Store == 102)].iloc[:,3:].values
plt.plot(a[0])
plt.show()

In [ ]:
#Get ARIMA output

ARIMA_dir = os.path.join(data_dir, 'fake_ARIMA_0109.txt')
ARIMA_df_raw = pd.read_csv(ARIMA_dir, sep = ',', header = 0, index_col=0).sort_values(by=['SKU', 'Store', 'Week'])
ARIMA_df_raw.reset_index(drop=True)
STR_SKU_LIST = ARIMA_df_raw.loc[:, ['SKU', 'Store']].drop_duplicates().copy().reset_index(drop=True)

ARIMA_output = ARIMA_df_raw['Units'].values.reshape(-1,52)
ARIMA_h = pd.DataFrame(ARIMA_output, columns = [i for i in range(1,53)])
ARIMA_df = pd.concat([STR_SKU_LIST, ARIMA_h], axis=1)
ARIMA_df.head()

In [ ]:
kmean_fcst = get_Kmeans_fcst(ARIMA_df, df_result)
# kmean_fcst.to_csv(os.path.join(data_dir, 'Output/final_forecast_0109.csv'))
kmean_fcst

In [ ]:
def plot_sls(df, *num_plot):
    if not num_plot:
        for i in range(np.size(df, 0)):
            plt.plot(df.iloc[i, 2:].values)
    else:
        rand_int = np.random.randint(0, np.size(df, 0), num_plot[0])
        plot_df = df.iloc[rand_int, :]
        plot_data = plot_df.iloc[:, 2:]
        item_loc = plot_df.iloc[:,0:2]

        for i in range(num_plot[0]):
            plt.plot(plot_data.iloc[i].values)
    
    ticks = np.arange(0, 52, 5)
    labels = np.arange(1, 53, 5)
    plt.xticks(ticks, labels)
    print (item_loc)
    plt.show()
    
    
def plot_sales_fcst(SKU, Store):
    sales_2014 = year1_df[(year1_df.SKU == SKU) & (year1_df.Store == Store)].iloc[:,2:]
    sales_2015 = year2_df[(year2_df.SKU == SKU) & (year2_df.Store == Store)].iloc[:,2:]
    sales_2016 = year3_df[(year3_df.SKU == SKU) & (year3_df.Store == Store)].iloc[:,2:]
    fcst = kmean_fcst[(kmean_fcst.SKU == SKU) & (kmean_fcst.Store == Store)].iloc[:,2:]
    
    plt.figure(figsize=(12, 9))
    sales2014, = plt.plot(sales_2014.iloc[0].values, label = 'sales2014')
    sales2015, = plt.plot(sales_2015.iloc[0].values, label = 'sales2015')
    sales2016, = plt.plot(sales_2016.iloc[0].values, label = 'sales2016')
    fcst, = plt.plot(fcst.iloc[0].values, label = 'fcst')
    ticks = np.arange(0, 52, 5)
    labels = np.arange(1, 53, 5)
    plt.xticks(ticks, labels)
    plt.legend(handles=[sales2014, sales2015, sales2016, fcst])
    
    plt.show()

In [ ]:
#plot sales hist and fcst for a SKU, Store
plot_sales_fcst(999, 109)

In [ ]:
plot_sls(kmean_fcst,2)